# Hooks

Agent with lifecycle hooks for monitoring and debugging

## Imports

In [ ]:
from strands import Agent
from strands.models.openai import OpenAIModel
from strands.hooks.events import BeforeInvocationEvent, AfterInvocationEvent
from strands.hooks import HookProvider, HookRegistry
from strands_tools import calculator

## Setup Local LLM Client

In [ ]:
model = OpenAIModel(
    client_args={
        "api_key": "lm-studio",
        "base_url": "http://localhost:1234/v1",
    },
    model_id="local-model",
    params={
        "temperature": 0.1,
        "top_p": 0.9,
    }
)

## Create Agent

In [ ]:
agent = Agent(
    model=model,
    tools=[calculator],
    system_prompt="You are a helpful assistant."
)

## Setup Simple Hooks

In [ ]:
def before_callback(event: BeforeInvocationEvent) -> None:
    print("Before invocation")

def after_callback(event: AfterInvocationEvent) -> None:
    print("After invocation")

agent.hooks.add_callback(BeforeInvocationEvent, before_callback)
agent.hooks.add_callback(AfterInvocationEvent, after_callback)

## Run Agent with Simple Hooks

In [ ]:
response = agent("Calculate 5 * 8")
print(response)

## Hook Provider Class

In [ ]:
class LoggingHook(HookProvider):
    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(BeforeInvocationEvent, self.log_start)
        registry.add_callback(AfterInvocationEvent, self.log_end)

    def log_start(self, event: BeforeInvocationEvent) -> None:
        print(f"Request started for agent: {event.agent.name}")

    def log_end(self, event: AfterInvocationEvent) -> None:
        print(f"Request completed for agent: {event.agent.name}")

## Agent with Hook Provider

In [ ]:
agent_with_provider = Agent(
    model=model,
    tools=[calculator],
    hooks=[LoggingHook()],
    system_prompt="You are a helpful assistant."
)

## Run Agent with Hook Provider

In [ ]:
response = agent_with_provider("Calculate 10 / 2")
print(response)